## Driverless AI - Python Scoring Pipeline 実行サンプル（クライアント環境（HTTPサーバへの接続））

HTTPサーバ(スコアリングフォルダ(scoring-pipeline)のrun_http_server.sh(http_server.pyの実行ファイル)を実行)へ、クライアントから接続

---

### コマンドラインでの実行
run_http_client.sh参照

#### 一行データのスコアリング

```bash
curl  52.196.26.99:9090/rpc  --header "Content-Type: application/json"  --data @- <<EOF
{
  "id": 1,
  "method": "score",
  "params": {
    "row": {"x1": "2.60", "x2": "-3.21", "x3": "0.62", "x4": "1.36"}
  }
}
EOF
```

> {"jsonrpc": "2.0", "id": 1, "result": [2.561763286590576]}

#### 複数行データのバッチスコアリング

```bash
curl  52.196.26.99:9090/rpc  --header "Content-Type: application/json"  --data @- <<EOF
{
  "id": 1,
  "method": "score_batch",
  "params": {
    "rows":[
         {"x1": "2.60", "x2": "-3.21", "x3": "0.62", "x4": "1.36"},
         {"x1": "0", "x2": "10", "x3": "0.3", "x4": "1"},
         {"x1": "-1.11", "x2": "-1.11", "x3": "2.22", "x4": "2.22"}
    ]
  }
}
EOF
```

> {"jsonrpc": "2.0", "id": 1, "result": [[2.561763286590576], [3.8429617881774902], [2.533003807067871]]}

---

### Python "Requests"での実行

[Requests](https://requests-docs-ja.readthedocs.io/en/latest/)

In [160]:
import requests
import json

In [161]:
headers = { 'Content-Type': 'application/json'}

data = {
  "id": 1,
  "method": "score",
  "params": {
    "row": {"x1": "2.60", "x2": "-3.21", "x3": "0.62", "x4": "1.36"}
  }
}
data = json.dumps(data)   # dictをstr型へ変換

In [162]:
data

'{"id": 1, "method": "score", "params": {"row": {"x1": "2.60", "x2": "-3.21", "x3": "0.62", "x4": "1.36"}}}'

In [163]:
response = requests.post('http://13.113.192.151:9090/rpc', headers=headers, data=data)
response

<Response [200]>

In [164]:
#response.headers

In [165]:
# 結果をjsonで取得
response.json()

{'jsonrpc': '2.0', 'id': 1, 'result': [2.561763286590576]}

In [166]:
response.text

'{"jsonrpc": "2.0", "id": 1, "result": [2.561763286590576]}'

#### ファイルからの実施

In [167]:
import pandas as pd

In [168]:
df = pd.read_csv('sample_data.csv')
df

,x1,x2,x3,x4
0,2.60,-3.21,0.62,1.36
1,0.00,10.00,0.30,1.00
2,-1.11,-1.11,2.22,2.22


In [169]:
# データを文字型に変換
df_str = df.astype(str)
df_str.dtypes

x1    object
x2    object
x3    object
x4    object
dtype: object

In [170]:
# pandas.DataFrameから、jsonに変換
data_json = df_str.to_json(orient="records")
data_json = json.loads(data_json)
data_json

[{'x1': '2.6', 'x2': '-3.21', 'x3': '0.62', 'x4': '1.36'},
 {'x1': '0.0', 'x2': '10.0', 'x3': '0.3', 'x4': '1.0'},
 {'x1': '-1.11', 'x2': '-1.11', 'x3': '2.22', 'x4': '2.22'}]

In [171]:
data = {
  "id": 1,
  "method": "score_batch",
  "params": {
    "rows": data_json
  }
}
data = json.dumps(data)   # dictをstr型へ変換
data

'{"id": 1, "method": "score_batch", "params": {"rows": [{"x1": "2.6", "x2": "-3.21", "x3": "0.62", "x4": "1.36"}, {"x1": "0.0", "x2": "10.0", "x3": "0.3", "x4": "1.0"}, {"x1": "-1.11", "x2": "-1.11", "x3": "2.22", "x4": "2.22"}]}}'

In [172]:
response = requests.post('http://13.113.192.151:9090/rpc', headers=headers, data=data)
response

<Response [200]>

In [173]:
response.json()

{'jsonrpc': '2.0',
 'id': 1,
 'result': [[2.561763286590576], [3.8429617881774902], [2.533003807067871]]}

In [174]:
y = [i[0] for i in response.json()['result']]
df['y'] = y
df

,x1,x2,x3,x4,y
0,2.60,-3.21,0.62,1.36,2.561763
1,0.00,10.00,0.30,1.00,3.842962
2,-1.11,-1.11,2.22,2.22,2.533004
